# script to prepare the files for expression QTL analysis

genotypes are only available through protected access

QTL was generated using QTLtools

In [2]:
import os
import pandas as pd
import numpy as np
import plotly.express as px
os.chdir("/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis")
base_dir = "/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis"

## prepare the normalized counts for eQTL

In [3]:
RNA_normalized_counts = pd.read_csv(f"{base_dir}/RNA_seq_analysis/RNA_normalized_counts.csv")
metadata_RNA = pd.read_csv(f"{base_dir}/metadata/cleaned_RNA_metadata.csv", index_col=0)
gtf_annotation_df = pd.read_pickle(f"{base_dir}/gencode_gtf.pickle")
gtf_transcripts = gtf_annotation_df[(gtf_annotation_df["feature"] == "transcript") & (gtf_annotation_df["transcript_type"] == "protein_coding")].dropna(axis=1, how='all')
gtf_transcripts["gene_id"] = gtf_transcripts["gene_id"].str.split(".").str[0]
gtf_transcripts["transcript_id"] = gtf_transcripts["transcript_id"].str.split(".").str[0]
gtf_transcripts["TSS_start"] = gtf_transcripts.apply(lambda x: int(x["start"]) if x["strand"] == "+" else int(x["end"]) ,axis = 1)
gtf_genes = gtf_annotation_df[(gtf_annotation_df["feature"] == "gene") & (gtf_annotation_df["gene_type"] == "protein_coding")].dropna(axis=1, how='all')
gtf_genes["gene_id"] = gtf_genes["gene_id"].str.split(".").str[0]
gtf_genes["TSS_start"] = gtf_genes.apply(lambda x: int(x["start"]) if x["strand"] == "+" else int(x["end"]) ,axis = 1)
gtf_genes["length"] = gtf_genes["end"].astype(int) - gtf_genes["start"].astype(int)

In [4]:
# filter for protein coding genes
RNA_normalized_counts = RNA_normalized_counts.merge(gtf_genes[["gene_id","seqname","TSS_start", "length", "strand"]].drop_duplicates("gene_id"), left_on = "ENSG", right_on = "gene_id")
RNA_normalized_counts["TSS_end"] = RNA_normalized_counts["TSS_start"] + 1 

In [16]:
# we don't have genotypes for NRHV319 and NRHV005
samples = metadata_RNA[~metadata_RNA["patient"].isin(["NRHV319", "NRHV005"])]["patient"].to_list()

## CD8 counts

In [17]:
wanted_samples = metadata_RNA[(metadata_RNA["cell_type"] == "CD8") & metadata_RNA["patient"].isin(samples)]["sample"].to_list()

In [18]:
counts_CD8 = RNA_normalized_counts[["seqname", "TSS_start", "TSS_end", "gene_id", "length", "strand"] + wanted_samples].copy()

In [19]:
df_mapper = metadata_RNA[(metadata_RNA["cell_type"] == "CD8") & metadata_RNA["patient"].isin(samples)][["sample", "patient"]]

In [20]:
counts_CD8[wanted_samples] = np.log2(counts_CD8[wanted_samples] + 1)

In [21]:
counts_CD8 = counts_CD8.rename(columns = dict(zip(df_mapper['sample'], df_mapper['patient'])))
counts_CD8 = counts_CD8.rename(columns = {"seqname":"#chr", "TSS_start":"start", "TSS_end":"end","gene_id":"gene"})

In [26]:
counts_CD8.sort_values(["#chr","start"]).to_csv("/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/RNA/RNA_expr_CD8_test.bed", sep = "\t", index = False)
!bgzip -f /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/RNA/RNA_expr_CD8_test.bed
!tabix -f -p bed /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/RNA/RNA_expr_CD8_test.bed.gz

## CD4 counts

In [28]:
# We have also identified PSA4956 and NRHV171 to be outliers
wanted_samples = metadata_RNA[(metadata_RNA["cell_type"] == "CD4") & metadata_RNA["patient"].isin(samples) & (~metadata_RNA["patient"].isin(["PSA4956", "NRHV171"]))]["sample"].to_list()

In [29]:
counts_CD4 = RNA_normalized_counts[["seqname", "TSS_start", "TSS_end", "gene_id", "length", "strand"] + wanted_samples].copy()

In [30]:
df_mapper = metadata_RNA[(metadata_RNA["cell_type"] == "CD4") & metadata_RNA["patient"].isin(samples)][["sample", "patient"]]

In [31]:
counts_CD4[wanted_samples] = np.log2(counts_CD4[wanted_samples] + 1)

In [32]:
counts_CD4 = counts_CD4.rename(columns = dict(zip(df_mapper['sample'], df_mapper['patient'])))
counts_CD4 = counts_CD4.rename(columns = {"seqname":"#chr", "TSS_start":"start", "TSS_end":"end","gene_id":"gene"})

In [33]:
counts_CD4.sort_values(["#chr","start"]).to_csv("/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/RNA/RNA_expr_CD4_test.bed", sep = "\t", index = False)
!bgzip -f /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/RNA/RNA_expr_CD4_test.bed
!tabix -f -p bed /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/RNA/RNA_expr_CD4_test.bed.gz

## running a permutation test to identify the best number of PCA component as covariates identified the following:
for CD8: 0 genotype covariates and 5 phenotype covariates
for CD4: 0 genotype covariates and 7 phenotype covariates

In [ ]:
# see bash scripts for running of QTL tools

In [24]:
# merge the final results
!cat /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/RNA/output_final/RNA_nominal_CD4_0* > /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/RNA/output_final/RNA_nominal_CD4_merged.txt
!cat /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/RNA/output_final/RNA_nominal_CD8_0* > /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/RNA/output_final/RNA_nominal_CD8_merged.txt

!cat /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/RNA/output_final/RNA_permuted_CD4_0* > /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/RNA/output_final/RNA_permuted_CD4_merged.txt
!cat /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/RNA/output_final/RNA_permuted_CD8_0* > /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/RNA/output_final/RNA_permuted_CD8_merged.txt

In [5]:
RNA_permuted_CD8 = pd.read_csv("/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/RNA/output_final/RNA_permuted_CD8_merged.txt", sep = " ")
len(RNA_permuted_CD8[RNA_permuted_CD8["adj_beta_pval"] < 0.1])

3610

In [6]:
RNA_permuted_CD4 = pd.read_csv("/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/RNA/output_final/RNA_permuted_CD4_merged.txt", sep = " ")
len(RNA_permuted_CD4[RNA_permuted_CD4["adj_beta_pval"] < 0.1])

3150